# YOHO training

In [2]:
# Import used libraries

import pandas as pd
print("Pandas version: ", pd.__version__)

import torch
torch.manual_seed(0)

from utils import AudioFile, YOHODataset, YOHODataGenerator

Pandas version:  2.1.2


## Data generator

In [3]:
urbansed_df = YOHODataset(
    audios=[
        AudioFile(filepath=file.filepath, labels=file.events)
        for _, file in pd.read_csv("./data/urbansed.csv").iterrows()
    ]
)

print("x_0: ", urbansed_df[0][0])
print("y_0: ", urbansed_df[0][1])

x_0:  tensor([[[-0.0329,  0.4425,  0.5799,  ...,  1.3928, -0.0962, -0.2009],
         [-0.0348,  0.4374,  0.5723,  ...,  1.3889, -0.0655, -0.2012],
         [-0.0381,  0.4289,  0.5596,  ...,  1.3824, -0.0217, -0.2018],
         ...,
         [-1.1914, -1.4997, -1.3121,  ..., -1.9538, -1.2554, -2.2680],
         [-1.2754, -1.5072, -1.2038,  ..., -1.7031, -1.2814, -2.9766],
         [-1.3882, -1.6844, -1.2313,  ..., -1.4122, -1.3381, -2.5239]]])
y_0:  [('noise', 0, 10.0), ('jackhammer', 3.1254216166103967, 4.081540068079489), ('dog_bark', 4.272822510637965, 5.995148792556066), ('drilling', 4.644565713394142, 7.674565713394141)]


In [9]:
urbansed_df[2][0]

tensor([[[-0.5876, -0.1638, -0.0413,  ...,  0.5710, -0.1948, -2.6545],
         [-0.5894, -0.1683, -0.0481,  ...,  0.5662, -0.1982, -2.5735],
         [-0.5922, -0.1759, -0.0594,  ...,  0.5582, -0.2039, -2.4757],
         ...,
         [-1.6205, -1.8953, -1.7280,  ..., -2.7272, -1.7148, -1.9623],
         [-1.6954, -1.9020, -1.6315,  ..., -2.2517, -1.7181, -1.9873],
         [-1.7960, -2.0600, -1.6561,  ..., -1.9169, -1.7318, -1.8805]]])

In [5]:
train_dataloader = YOHODataGenerator(urbansed_df, batch_size=1, shuffle=True)

In [6]:
train_features, train_labels = next(iter(train_dataloader))
train_features, train_labels

(tensor([[[[-0.5202, -0.1274, -0.0139,  ...,  0.8030, -1.2471, -0.9547],
           [-0.5218, -0.1316, -0.0202,  ...,  0.7998, -1.0589, -0.9574],
           [-0.5245, -0.1386, -0.0306,  ...,  0.7945, -0.8947, -0.9616],
           ...,
           [-1.4776, -1.7323, -1.5773,  ..., -0.6732, -0.8195, -0.9068],
           [-1.5470, -1.7385, -1.4878,  ..., -0.7840, -1.0408, -0.9202],
           [-1.6402, -1.8850, -1.5106,  ..., -0.9398, -1.1218, -1.0222]]]]),
 ("[('noise', 0, 10.0), ('street_music', 2.746415593304119, 5.572560610897536), ('jackhammer', 3.578242701949975, 5.962773700434363), ('car_horn', 4.852808255154279, 7.616477860260049), ('gun_shot', 5.596996277018889, 6.324215277018888), ('jackhammer', 6.076060595610772, 7.614903168559016), ('dog_bark', 6.710908004347111, 7.732751405172891), ('siren', 7.085406683277, 9.5606704776191), ('gun_shot', 7.210237848455257, 9.609737848455257), ('gun_shot', 8.03668790669237, 9.80668790669237)]",))

In [7]:
train_features, train_labels = next(iter(train_dataloader))
train_features, train_labels

(tensor([[[[-0.1787,  0.2099,  0.3221,  ...,  0.9875, -0.2363, -0.3157],
           [-0.1803,  0.2057,  0.3159,  ...,  0.9843, -0.2106, -0.3160],
           [-0.1829,  0.1988,  0.3056,  ...,  0.9791, -0.1743, -0.3164],
           ...,
           [-1.1256, -1.3775, -1.2242,  ..., -1.7420, -1.1733, -2.0069],
           [-1.1942, -1.3836, -1.1357,  ..., -1.5353, -1.1901, -2.0069],
           [-1.2864, -1.5285, -1.1582,  ..., -1.3005, -1.2324, -2.0069]]]]),
 ("[('noise', 0, 10.0), ('siren', 2.1688161435155937, 3.708788645004307), ('engine_idling', 4.957180390568066, 5.815470695198912), ('engine_idling', 5.132277516634661, 6.33779061494837), ('jackhammer', 5.399908538681259, 6.027716925087238), ('street_music', 9.249252610765392, 9.833070734254035)]",))